In [61]:
# Импортирование основных библиотек

! pip install gensim
! pip install gensim --upgrade
import pandas as pd, os
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec,KeyedVectors
from gensim import downloader
import numpy as np
import datasets
import gensim
import nltk

In [62]:
# Импортирование датасета

dataset = pd.read_csv('/kaggle/input/product-item-data/sample-data.csv')
dataset

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."
...,...,...
495,496,Cap 2 bottoms - Cut loose from the maddening c...
496,497,Cap 2 crew - This crew takes the edge off fick...
497,498,All-time shell - No need to use that morning T...
498,499,All-wear cargo shorts - All-Wear Cargo Shorts ...


In [63]:
# Выделение из датасета колонки с текстами

train_text_df = pd.DataFrame({'text': dataset['description']})
train_text_df.head()

,text
0,Active classic boxers - There's a reason why o...
1,Active sport boxer briefs - Skinning up Glory ...
2,Active sport briefs - These superbreathable no...
3,"Alpine guide pants - Skin in, climb ice, switc..."
4,"Alpine wind jkt - On high ridges, steep ice an..."


In [64]:
# Приготовление эмбеддингов tf_idf

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings_tf_idf = tfidf.fit_transform(train_text_df['text'] ).toarray()
text_embeddings_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
embeddings = gensim.downloader.load('word2vec-google-news-300')

In [70]:
text_embeddings_word2vec = []
stopwords = nltk.corpus.stopwords.words('english')
for doc in train_text_df['text'].str.lower().str.replace('[a-z]', ''):
    temp = []
    for word in doc.split(' '):
        if word not in stopwords:
            try:
                word_vec = embeddings[word]
                temp.append(word_vec)
            except:
                pass
    doc_vector = sum(temp)/len(temp)
    text_embeddings_word2vec.append(doc_vector)
len(text_embeddings_word2vec)

500

In [94]:
# Матрица косинусных расстояний для эмбеддингов tf_idf

array = sklearn.metrics.pairwise.cosine_similarity(text_embeddings_tf_idf, Y=None, dense_output=True)
array

array([[1.        , 0.18652085, 0.11989642, ..., 0.04386885, 0.09243105,
        0.07568186],
       [0.18652085, 1.        , 0.43452928, ..., 0.0407376 , 0.12165498,
        0.05534363],
       [0.11989642, 0.43452928, 1.        , ..., 0.02476351, 0.05161329,
        0.03228341],
       ...,
       [0.04386885, 0.0407376 , 0.02476351, ..., 1.        , 0.04610441,
        0.0454197 ],
       [0.09243105, 0.12165498, 0.05161329, ..., 0.04610441, 1.        ,
        0.31195031],
       [0.07568186, 0.05534363, 0.03228341, ..., 0.0454197 , 0.31195031,
        1.        ]])

In [71]:
# Матрица косинусных расстояний для эмбеддингов word2vec

array2 = sklearn.metrics.pairwise.cosine_similarity(text_embeddings_word2vec, Y=None, dense_output=True)
array2

array([[0.9999997 , 0.9032341 , 0.8529645 , ..., 0.8572532 , 0.82991874,
        0.85092926],
       [0.9032341 , 1.0000005 , 0.9238016 , ..., 0.86417127, 0.84152335,
        0.8347699 ],
       [0.8529645 , 0.9238016 , 1.0000001 , ..., 0.85212815, 0.8097455 ,
        0.8188008 ],
       ...,
       [0.8572532 , 0.86417127, 0.85212815, ..., 0.9999999 , 0.8340199 ,
        0.8108154 ],
       [0.82991874, 0.84152335, 0.8097455 , ..., 0.8340199 , 1.0000002 ,
        0.9040051 ],
       [0.85092926, 0.8347699 , 0.8188008 , ..., 0.8108154 , 0.9040051 ,
        0.99999976]], dtype=float32)

In [95]:
# Уровень значимости косинусного расстояния для эмбеддингов tf_idf: 0.4

level = 0.4

In [77]:
similar_text_list_tf_idf = []
for i in range(len(array)):
    lst_a = []
    for j in range(i,len(array)):
        x = array[i][j]
        if x > level:
            lst_a.append(j)
    similar_text_list_tf_idf.append(lst_a)

similar_text_list_tf_idf

[[0],
 [1, 2],
 [2],
 [3, 158],
 [4, 307],
 [5],
 [6],
 [7, 218, 219, 253, 254, 260, 261, 290],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14, 15, 479, 483, 484],
 [15, 484],
 [16],
 [17, 20, 170, 171, 340, 486],
 [18, 493],
 [19, 20, 171, 339, 340, 486, 487],
 [20, 170, 171, 339, 340, 486, 487],
 [21, 22, 173, 174, 358, 359, 495, 496],
 [22, 23, 173, 174, 358, 359, 495, 496],
 [23, 174, 276, 439, 440, 441, 442],
 [24, 175, 276, 445],
 [25, 35, 67, 97, 490],
 [26, 27, 451, 452],
 [27, 451, 452],
 [28],
 [29],
 [30],
 [31, 461, 462],
 [32],
 [33],
 [34, 178, 281, 472],
 [35, 490],
 [36, 480, 481],
 [37],
 [38],
 [39],
 [40],
 [41, 420],
 [42, 426],
 [43, 426],
 [44],
 [45, 408],
 [46],
 [47, 68, 237, 318, 331, 370, 395, 489],
 [48, 133],
 [49, 438],
 [50],
 [51, 443],
 [52, 341],
 [53],
 [54],
 [55, 396],
 [56],
 [57, 62, 63, 64, 431, 432, 466, 469, 475],
 [58],
 [59],
 [60],
 [61],
 [62, 63, 64, 431, 432, 466, 469, 470, 475, 490],
 [63, 64, 431, 432, 466, 469, 475],
 [64, 72, 431, 432, 

In [78]:
# Уровень значимости косинусного расстояния для эмбеддингов word2vec: 0.95

level2 = 0.95

In [79]:
# Cоздание списка похожих текстов для эмбеддингов word2vec. 
# В подсписках списка - тексты, похожие на соответствующий текст (идёт первым в подсписке, нумерация с 0).

similar_text_list_word2vec = []
for i in range(len(array2)):
    lst_a = []
    for j in range(i,len(array2)):
        x = array2[i][j]
        if x > level2:
            lst_a.append(j)
    similar_text_list_word2vec.append(lst_a)

similar_text_list_word2vec

[[0],
 [1],
 [2],
 [3, 158],
 [4, 307],
 [5, 183],
 [6],
 [7, 219],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14, 15, 479, 483, 484],
 [15, 479, 483, 484],
 [16],
 [17, 170],
 [18, 493],
 [19, 171, 339, 486, 487],
 [20, 171, 340, 487],
 [21, 22, 173, 174, 358, 359, 495, 496],
 [22, 173, 174, 358, 359, 496],
 [23, 440, 442],
 [24, 175, 276, 445],
 [25],
 [26, 27, 451, 452],
 [27, 451, 452],
 [28, 453],
 [29],
 [30, 436],
 [31, 461, 462],
 [32, 50],
 [33],
 [34, 281],
 [35],
 [36, 480, 481],
 [37],
 [38],
 [39],
 [40],
 [41, 420],
 [42, 426],
 [43, 426],
 [44],
 [45, 408],
 [46],
 [47, 68, 237, 318, 395, 489],
 [48, 133],
 [49, 438],
 [50, 128],
 [51, 443],
 [52, 341],
 [53],
 [54],
 [55],
 [56],
 [57, 62, 63, 64, 431, 432],
 [58],
 [59],
 [60],
 [61],
 [62, 63, 64, 431, 432],
 [63, 64, 431, 432],
 [64, 431, 432],
 [65],
 [66],
 [67],
 [68, 237, 318, 331, 395, 489],
 [69],
 [70],
 [71, 333],
 [72, 238, 334],
 [73],
 [74, 76, 360],
 [75, 124],
 [76, 360],
 [77],
 [78, 79],
 [79],
 [80, 45

In [93]:
with open("Predictions.txt", "w") as file:
    file.write("TF_IDF PREDICTIONS:")
    file.write('\n')
    for i in similar_text_list_tf_idf:
        file.write(str(i))
        file.write('\n')
    file.write('\n')
    file.write("WORD2VEC PREDICTIONS:")
    file.write('\n')
    for i in similar_text_list_word2vec:
        file.write(str(i))
        file.write('\n')